# Finite volume discretization of unsteady two-dimensional advection-diffusion equation

The equation we would like to solve numerically is the following:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_j \frac{\partial \phi}{\partial x_j} = \frac{\partial}{\partial x_j} \biggl(\Gamma \frac{\partial\phi}{\partial x_j}\biggr) + S $$

In two-dimensions the partial derivatives are expanded as:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_x \frac{\partial \phi}{\partial x} + \rho u_y \frac{\partial \phi}{\partial y} = \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial\phi}{\partial x}\biggr) +\frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial\phi}{\partial y}\biggr) + S $$


In what follows the source term is taken to be zero $S=0$:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_x \frac{\partial \phi}{\partial x} + \rho u_y \frac{\partial \phi}{\partial y} = \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial\phi}{\partial x}\biggr) +\frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial\phi}{\partial y}\biggr) $$

## Unsteady purely conduction equation

We will start with unsteady purely conduction equation:
$$\rho \frac{\partial \phi}{\partial t} = \frac{\partial}{\partial x}\biggl(\Gamma \frac{\partial \phi}{\partial x}\biggr) + \frac{\partial}{\partial y}\biggl(\Gamma \frac{\partial \phi}{\partial y}\biggr)$$

Let's integrate this equation over the control volume:
$$\rho \int_w^e\int_s^n \frac{\partial \phi}{\partial t} dx dy = \int_w^e\int_s^n \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial \phi}{\partial x}\biggr) dx dy + \int_w^e\int_s^n \frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial \phi}{\partial y}\biggr) dx dy$$

![alt text](integration.png "2D control volume")

The integration leads to the following:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t} \phi_P   = \Delta y_P \biggl(\Gamma_e \Bigl(\frac{\partial\phi}{\partial x}\Bigr)_e-\Gamma_w \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_w\biggr) + \Delta x_P \biggl(\Gamma_n \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_n-\Gamma_s \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_s\biggr)$$

Assume that the partial derivatives can be represented through forward difference:
$$\Bigl(\frac{\partial \phi}{\partial x}\Bigr)_e = \frac{\phi_E - \phi_P}{\Delta x_e}$$
$$\Bigl(\frac{\partial \phi}{\partial x}\Bigr)_w = \frac{\phi_P - \phi_W}{\Delta x_w}$$
$$\Bigl(\frac{\partial \phi}{\partial y}\Bigr)_n = \frac{\phi_N - \phi_P}{\Delta y_n}$$
$$\Bigl(\frac{\partial \phi}{\partial y}\Bigr)_s = \frac{\phi_P - \phi_S}{\Delta y_s}$$,
where $\Delta x_P = \Delta x_e + \Delta x_w$ and $\Delta y_P = \Delta y_n + \Delta y_s$


Thus, after substitution in the numerical equation the following is obtained:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t}\phi_P = \Delta y_P \biggl(\Gamma_e \frac{\phi_E - \phi_P}{\Delta x_e} - \Gamma_w \frac{\phi_P-\phi_W}{\Delta x_w}\biggr)+\Delta x_P \biggl(\Gamma_n \frac{\phi_N-\phi_P}{\Delta y_n}-\Gamma_s \frac{\phi_P - \phi_S}{\Delta y_s}\biggr)$$

Let's combine the terms accordingly:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t}\phi_P = \phi_P \biggl(-\Gamma_e\frac{\Delta y_P}{\Delta x_e}-\Gamma_w \frac{\Delta y_P}{\Delta x_w}-\Gamma_n \frac{\Delta x_P}{\Delta y_n} - \Gamma_s \frac{\Delta x_P}{\Delta y_s}\biggr) + \phi_E \Gamma_e \frac{\Delta y_P}{\Delta x_e} + \phi_W \Gamma_w \frac{\Delta y_P}{\Delta x_w} + \phi_S \Gamma_s \frac{\Delta x_P}{\Delta y_s} + \phi_N \Gamma_n \frac{\Delta x_P}{\Delta y_n}$$

Now the problem which we would like to solve is a slab with two constraints $\phi(x=0,y)=1$ and $\phi(x=L,y)=0$. As this problem is two-dimensional, we will impose two adiabatic boundary conditions on top and the bottom of the domain, i.e. $\frac{\partial \phi(x,y=0)}{\partial y}=0$ and $\frac{\partial \phi(x,y=H)}{\partial y}=0$. 

To derive boundary conditions we need to introduce the control volume near boundaries. The points are placed in the center of control volumes, so the boundary conditions are naturally involved.

![alt text](boundary_volumes.png?arg "Boundary control volumes")

Now, let's try to compose the matrix assuming the following quantities: $\rho = 1$, $\Gamma=1$, $L=1$, $H=2$. The numerical domain for points will be taken as $0..NX+1$ and $0..NY+1$, where $NX=10$ and $NY=20$. Then we will have $\Delta x = \Delta y = 0.1$. Notice that points close to boundaries will have only half distance of $0.5 \Delta x$ and $0.5 \Delta y$.

The matrix will be is as follows:
$$\begin{aligned}
&\phi_{0,0..NY+1}=0\\
&\phi_{NX+1,0..NY+1}=1\\
&\frac{\partial \phi_{1..NX,0}}{\partial y} = 0\\
&\frac{\partial \phi_{1..NX,NY+1}}{\partial y} = 0
\end{aligned}$$

For the bulk nodes the following equation will be fulfilled assuming $\rho$, $\Gamma$, $L$ and $H$ from above:
$$
\begin{aligned}
&10^{-2} \frac{\partial}{\partial t}\phi_{i,j} = -8 \phi_{i,j} + 2 \phi_{i-1,j} + 2 \phi_{i+1,j} + 2 \phi_{i,j-1} + 2 \phi_{i,j+1} &&\text{for }i=1..NX\text{ and }j=2..NY-1\\
&0.75\times 10^{-2} \frac{\partial}{\partial t}\phi_{i,1} = -6 \phi_{i,1} + 2 \phi_{i-1,1} + 2 \phi_{i+1,1} + 2 \phi_{i,2} &&\text{for }i=1..NX\\
&0.75\times 10^{-2} \frac{\partial}{\partial t}\phi_{i,NY} = -6 \phi_{i,NY} + 2 \phi_{i-1,NY} + 2 \phi_{i+1,NY} + 2 \phi_{i,NY-1} &&\text{for }i=1..NX\\
\end{aligned}$$

Though the system looks quite simple, the connection between coefficients between rows and columns doesn't give an opportunity to easily write the matrix. Instead, we will start numerically allocate the matrix:

In [2]:
import numpy
import pylab

In [4]:
NX=10
NY=20
phi = numpy.array((NY+2,NX+2))

In [5]:
mat = numpy.array(((NX+2)*(NY+2),(NX+2)*(NY+2)))